In [ ]:
%matplotlib notebook

import h5py
import matplotlib.pyplot as plt
from numpy import *
import pandas as pd
import os
import sys
from scipy.optimize import curve_fit
from typing import Dict, Any
import recordclass.recordclass as rc
from itertools import product

if (os.name == 'posix'):
    repo_path = r"/Volumes/C/Public/Users/Hybrid/Repos/Hybrid_H5/H5_python3"
else:
    repo_path = r"C:\Users\Hybrid\Repos\Hybrid_H5\H5_python3"
sys.path.append(repo_path)
# local imports
import HamamatsuH5
from Iterations import Iterations,seed_permute
from PlottingH5 import default_plotting, iterate_plot_2D
from FittingH5 import *

In [ ]:
def plt_subplots(nrows=1, ncols=1,*args,**kwargs):
    """
    Function to wrap plt.subplots and make sure the same block of code can plot many axes and a single axis
    """
    fig,axarr = plt.subplots(nrows,ncols,*args,**kwargs)
    if nrows == 1 and ncols == 1:
        axarr = [axarr]
    return fig,axarr
    
h5file = h5py.File('results.hdf5',mode='r+')

iterations = Iterations(h5file)
#Find number of measurements per experiment
measurements = h5file['settings/experiment/measurementsPerIteration'][()]+1
num_its = len(h5file['iterations'])

iterations
#Find independent variable names and values
#indep_vars = DataH5.get_indep_vars(h5file)
#iVars = list(indep_vars.keys())

# map iterations to independent variable values
# iterations = DataH5.make_iterations_df(h5file, iVars)

## Create n-darray of iteration numbers
* useful when there are multiple independent variables
* Indexing is [ivar1,ivar2,ivar3,...,ivarn]
* ivar ordering is alphabetical
* identical to ordering of keys in indep_vars

# Hamamatsu

In [ ]:
# Set ROI
fg = HamamatsuH5.set_frame_grabber_region(h5file)
width = fg['right']-fg['left']
height = fg['bottom']-fg['top']

# For documentation try > help(HMROI)
roi = HamamatsuH5.HMROI(width,height,dic = {
    "left" : 2,
    "right" : 5,
    "top" : 4,
    "bottom" : 7
})

#Load data into memory
shots_per_measurement = int(h5file['/settings/experiment/LabView/camera/shotsPerMeasurement/function'][()])
# pixel-by-pixel data indexed : [iteration, measurement, shot, y-pixel, x-pixel]
hm_pix = HamamatsuH5.load_data(h5file,roi)

#take pixel-by-pixel data and treat it into counter data then mean data
# "count" data indexed : [iteration, measurement, shot]
hm_counts = hm_pix.sum(3).sum(3)
# mean count data indexed : [iteration, shot]
mus = hm_counts.mean(1)
# standard deviation in mean indexed : [iteration, shot]
ers = hm_counts.std(1)/sqrt(measurements)

## Plot Histograms of count data

In [ ]:
plots = True
if plots and len(iterations) > 22:
    ans = input(f"You're about to plot {len(iterations)} histograms. Are you sure you want to do that? (y/N) : ").upper()[0]
    plots = ans=="Y"
if plots:
    fig,axarr = plt_subplots(len(iterations),1,figsize=(6,4*len(iterations)))
    for iteration, row in iterations.iterrows():
        iteration = int(iteration)
        for shot in range(shots_per_measurement):
            axarr[iteration].hist(hm_counts[iteration,:,shot],histtype='step', label = f"shot {shot}",bins=30)
        axarr[iteration].legend()
        axarr[iteration].set_title(str(row))
        fig.tight_layout()
    fig.show()
else:
    print("You have chosen not to plot the histograms.")

## Plot mean count data
* If only 1 iteration : Prints out means for each shot and the corresponding standard deviation
* If there is 1 independent variable : Plot means for each iteration with error bars
* If there are 2 independent variables : show image of means for each iteration.

In [ ]:
default_plotting(iterations,mus,ers,shots_per_measurement,description="Mean Counts in ROI")

## 2D as many 1D

* Often, when a 2D scan is performed it's more enlightening to plot each row (or column) of data as a line graph

In [ ]:
iterate_plot_2D(**{
    "iterations": iterations,
    "data": mus,
    "data_error": ers,
    "description": "Counts in ROI",
})

## Show images taken within ROI

In [ ]:
## show average image for one shot for each iteration
im_shot = 0 # shot to image

fig,axarr = plt_subplots(len(iterations),1,figsize=(6,4*len(iterations)))
for index, row in iterations.iterrows():
    iteration = row['iteration']
    axarr[iteration].imshow(hm_pix[iteration,:,im_shot,:,:].mean(0))
    axarr[iteration].set_title(str(row))
fig.tight_layout()
fig.show()

## Plot Histgram data of each picture for each iteration
* Very memory hungry!
* Please clear output before saving, copying or pushing if this cell was run

In [ ]:
im_width = roi.right-roi.left
im_height = roi.bottom-roi.top
for iteration, row in iterations.iterrows():
    i = row['iteration']
    fig,axarr = plt.subplots(im_height,im_width,figsize = (10,10))
    for y in range(im_height):
        for x in range(im_width):
            bns = 20
            for shot in range(shots_per_measurement):
                axarr[y,x].hist(hm_pix[i,:,shot,y,x],bins=bns,histtype='step',label = f"Shot {shot}")
            axarr[y,x].set_title(f"x = {x}, y = {y}")
            axarr[y,x].legend()
    fig.suptitle(str(row))
    fig.tight_layout()
    fig.show()

# Cuts

In [ ]:
count_data = hm_counts
shot_to_fit = 1

func = double_gaussian
guess = {
    "mu1": 22560,
    "mu2": 27000,
    "std1": 500,
    "std2": 2000,
    "a1": 2000,
    "a2": 1000
}

for i, row in iterations.iterrows():
    iteration = row['iteration']
    bns = 30
    hist, bin_edges = histogram(count_data[iteration,:,shot_to_fit],bins=bns)
    
    popt,pcov = curve_fit(f=func,xdata = bin_edges[1:], ydata = hist, p0 = list(guess.values()))
    perr = sqrt(diag(pcov))  # uncertainty in fit parameters
    
    cut = gauss_intercept(*popt)
    cut_err = cut_err_gauss(*append(popt,perr))
    
    print(f"cut_err: {cut_err}")
    cut_h = cut+cut_err
    cut_l = cut-cut_err
    
    for i,var in enumerate(guess):
        exec(f"{var} = {popt[i]}")
        exec(f"d{var} = {perr[i]}")
        
    
    ovlp = gauss_discrimination_error(cut,*popt)
    
    xln = linspace(min(count_data[iteration,:,shot_to_fit]),max(count_data[iteration,:,shot_to_fit]),1000)
    fig,ax = plt.subplots(1,1)
    ax.hist(count_data[iteration,:,1],bins=bns,histtype='step',label = f"Hist")
    ax.plot(xln,func(xln,*popt),label=f"Fit")
    ax.plot(xln,gaussian(xln,*popt[0::2]),label=f"0 atom")
    ax.plot(xln,gaussian(xln,*popt[1::2]),label=f"1 atom")
    ax.axvline(cut)
    ax.axvline(cut_l)
    ax.axvline(cut_h)
    ax.set_title(f"mu1 : {mu1:.0f} $\pm$ {dmu1:.0f} | mu2 : {mu2:.0f} $\pm$ {dmu2:.0f}\n cut : {cut:.0f} $\pm$ {cut_err:.0f}\n overlap: {ovlp:.3f}")
    ax.legend()
    fig.tight_layout()
    fig.show()

    print(f"Overlap : {ovlp}")
    print(f"cut : {cut}\ncut_err : {cut_err}")
    print(f"popt : {popt}\nperr : {perr}\npcov : {pcov}")

# Loading

In [ ]:
def shot_error(probability: float,samples: int):
    """
    Args:
        probability : number between 0 and 1. Measured probability of an event occurring 
        samples : number of samples used to measure the ratio
    Returns:
        shot noise uncertainty. Uncertainty in probability due to statistical variations
    """
    return sqrt(probability*(1-probability)/samples)

shots = 2

# Set count data variable, so this works regadless of device
count_data = hm_counts
cut = 24000  # 0 atom - 1 atom cut
multi_cut = 40000  # 1 atom - multi atom cut

zero_loading = (count_data < cut).sum(1)/measurements
zero_loading_error_bar = shot_error(zero_loading, measurements)
multi_loading = (count_data > multi_cut).sum(1)/measurements
multi_loading_error_bar = shot_error(multi_loading, measurements)
single_loading = 1-(zero_loading+multi_loading)
single_loading_error_bar = shot_error(single_loading, measurements)
multi_to_single_loading = multi_loading/single_loading
multi_to_single_loading_error_bar = sqrt(multi_loading_error_bar**2 + single_loading_error_bar**2)

loadings = {
    0: "Zero",
    1: "Single",
    2: "Multi",
    3: "Multi_to_Single"
}

for num,name in loadings.items():
    rate = eval(f"{name.lower()}_loading")
    rate_error_bar = eval(f"{name.lower()}_loading_error_bar")
    default_plotting(iterations,rate,rate_error_bar,shots=shots,description = f"{name} atom loading")

# Retention

In [ ]:
def shot_error(probability: float,samples: int):
    """
    Args:
        probability : number between 0 and 1. Measured probability of an event occurring 
        samples : number of samples used to measure the ratio
    Returns:
        shot noise uncertainty. Uncertainty in probability due to statistical variations
    """
    return sqrt(probability*(1-probability)/samples)

shots = 2

# Set count data variable, so this works regadless of device
count_data = hm_counts
cut = 24000  # 0 atom - 1 atom cut
multi_cut = 40000  # 1 atom - multi atom cut

single_loading = (count_data < multi_cut)*(count_data > cut)  # Measurements that were loaded
retained = single_loading[...,1]*single_loading[...,0]  # Measurements that were retained
loaded = single_loading[...,0].sum(1)  # Number of loaded measurements in each iteration

retention = retained.sum(1)/loaded
retention_error_bar = shot_error(retention,loaded)

default_plotting(iterations,retention,retention_error_bar,description = "Retention")

# Copy

In [ ]:
# Copy to next results folder

import datetime
import shutil
import re
import os
import operator
from numpy import sort

volume = os.getcwd()[0]
analysis_fname = 'Analysis3.ipynb'
measurement_path = os.path.dirname(os.path.dirname(os.getcwd()))
dir_filter = lambda f : os.path.isdir(f)

# If the latest experiment folder already has an analysis file, should that be overwritten?
override = True

# find latest day folder created by cspy
daily_folders = list(filter(
    dir_filter,
    [os.path.join(measurement_path,f) for f in os.listdir(measurement_path)]
))  # make sure non-folder names are not listed
today_path = sort(daily_folders)[-1]
print(today_path)
#find all experiments taken on the last day
experiments_today = list(filter(
    dir_filter,
    [os.path.join(today_path,f) for f in os.listdir(today_path)]
))
#and the last experiment taken the last day
last_experiment = sort(experiments_today)[-1]
exp_path = os.path.join(today_path, last_experiment)

# Copy
if analysis_fname not in os.listdir(exp_path) or override:
    print("Copying {} from :\n\t{}\nto:\n\t{}".format(analysis_fname,os.getcwd(),exp_path))
    shutil.copy(os.path.join(os.getcwd(),analysis_fname),exp_path)
else:
    print("Folder {} already have analysis file. If you wish to overwrite that file set 'override' to True".format(exp_path))

In [ ]:
# Copy to Git Repo

import datetime
import shutil
import re
import os
import operator
from numpy import sort

volume = os.getcwd()[0]
analysis_fname = 'Analysis3.ipynb'
measurement_path = os.path.dirname(os.path.dirname(os.getcwd()))
dir_filter = lambda f : os.path.isdir(f)

# If the latest experiment folder already has an analysis file, should that be overwritten?
override = True

# Copy
if analysis_fname not in os.listdir(repo_path) or override:
    print(f"Copying {analysis_fname} from :\n\t{os.getcwd()}\nto:\n\t{repo_path}")
    shutil.copy(os.path.join(os.getcwd(),analysis_fname),repo_path)
else:
    print(f"Folder {repo_path} already have analysis file. If you wish to overwrite that file set 'override' to True")